<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/5_5_22_1_politifact_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import glob
import json
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base_dir = Path("/content/drive/MyDrive/Research FakeNews")
assert base_dir.exists()

## Read data

In [6]:
%%capture
!unzip /content/drive/MyDrive/fakenews/Research\ FakeNews/Datasets\ Fakenews/fakenewsnet_dataset_politifact.zip

In [7]:
items = glob.glob(("/content/fakenewsnet_dataset/politifact/*/politifact*"), recursive=True)
len(items)

0

In [8]:
def _process_content(path:Path) -> dict:
    filepath = path/"news content.json"
    if filepath.exists():
        with open(filepath) as f:
            news_content = json.load(f)
    else:
        news_content = dict()
    return news_content

def _process_tweet(filepath:Path) -> dict:
    with open(filepath) as f:
        tweet = json.load(f)
    keys = ["id", "text", "retweet_count"]
    tweet_info = {k:tweet.get(k, None) for k in keys}
    user_keys = ["id", 'location', "friends_count", "followers_count"]
    user = tweet.get("user", {})
    user_info = {f"user_{k}":user.get(k, None) for k in user_keys}
    return {**tweet_info, **user_info}

In [14]:
for e in items:
    tweets = list((Path(e)/"tweets").glob("*.json"))
    if len(tweets):
        break

In [15]:
tweets[0]

NameError: ignored

In [13]:
_process_tweet(tweets[2])

NameError: ignored

In [16]:
base_keys = ["id", "label"]
news_content_keys = ['url', 'text', 'top_img', 'keywords', 'authors', 'canonical_link', 'title', 'meta_data', 'publish_date', 'source', 'summary']
tweeter_keys = ["retweets", "tweets"]
row_list = []

for e in tqdm(items):
    row = {}
    path = Path(e)
    row["id"] = path.name
    row["label"] = path.parent.name
    # add news content
    news_content = _process_content(path)
    for k in news_content_keys:
        row[k] = news_content.get(k, None)
    # add tweeter data
    row["retweets"] = [x.name.split(".")[0] for x in (path/"retweets").glob("*.json")]
    tweets = [_process_tweet(x) for x in (path/"tweets").glob("*.json")]
    row["tweets"] = tweets

    row_list.append(row)
    

df = pd.DataFrame(row_list, columns=base_keys+news_content_keys+tweeter_keys)

0it [00:00, ?it/s]

In [18]:
df.head()

,id,label,url,text,top_img,keywords,authors,canonical_link,title,meta_data,publish_date,source,summary,retweets,tweets


In [19]:
def get_tid(a):
    if len(a) == 0:
        return
    return a[0]["id"]


df.tweets.map(get_tid).isna().all()

True

In [20]:
(df.label == 'fake').sum(), (df.label == 'real').sum()

(0, 0)

In [21]:
len(df[df.text.str.len()==0]) + df.text.isna().sum()

0

## Load data

In [22]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/"politifact.csv", index_col=0, converters=converters, parse_dates=["publish_date"])

In [23]:
df.head()

,index,id,label,url,text,top_img,keywords,authors,canonical_link,title,meta_data,publish_date,source,summary,retweets,tweets
0,0,politifact1212,real,http://www.youtube.com/watch?v=5zrsl8o4ZPo&fea...,ein Google-Unternehmen\n\nDienste anbieten und...,http://www.google.com/favicon.ico,[],[],NaN,Bevor Sie zu YouTube weitergehen,"{'viewport': 'initial-scale=1, maximum-scale=5...",NaN,http://www.youtube.com,NaN,"[1022546912858103808, 1020337613474009089, 105...",[]
1,1,politifact6730,real,http://www.motherjones.com/politics/2012/09/se...,"During a private fundraiser earlier this year,...",https://www.motherjones.com/wp-content/uploads...,[],"['David Corn', 'Dave Gilson', 'Tim Murphy', 'B...",https://www.motherjones.com/politics/2012/09/s...,SECRET VIDEO: Romney Tells Millionaire Donors ...,"{'viewport': 'width=device-width, initial-scal...",1347904858.0,http://www.motherjones.com,NaN,"[873326045058838528, 872593394324951042, 87171...","[{'id': 873326045058838528, 'text': 'Liberal W..."
2,2,politifact2298,real,https://web.archive.org/web/20050322064340/htt...,"COPYRIGHT © 2005 LexisNexis, a division of Ree...",NaN,[],[],NaN,LexisNexis(R) Publisher,{},NaN,https://web.archive.org,NaN,"[963508344026693633, 962788790338174976, 96412...",[]
3,3,politifact87,real,http://www.ilga.gov/legislation/BillStatus.asp...,×\n\nThe Illinois General Assembly offers the ...,http://www.ilga.gov/LISlogo1.ico,[],[],NaN,Illinois General Assembly,"{'classification': 'Government', 'distribution...",NaN,http://www.ilga.gov,NaN,"[909138789754441730, 906251398718910464, 88211...",[]
4,4,politifact3180,real,http://abcnews.go.com/Politics/rand-paul-repub...,"Feb. 4, 2011  -- In an exclusive interview wi...",http://abcnews.go.com/Politics/rand-paul-repub...,[],"['Abc News', 'Jonathan Karl', 'February']",https://abcnews.go.com/Politics/rand-paul-repu...,ABC News Exclusive: Rand Paul Says Republicans...,{'description': 'In an exclusive interview wit...,NaN,http://abcnews.go.com,NaN,"[863760194244759552, 863421319752364032, 86376...",[]


In [24]:
num_retweets = df.retweets.map(len)
num_retweets.min(), num_retweets.mean(), num_retweets.max()

(0, 549.7300380228137, 29060)

In [25]:
num_tweets = df.tweets.map(len)
num_tweets.min(), num_tweets.mean(), num_tweets.max()

(0, 59.851711026615966, 21984)

## Add data for entries missing `news_content`

In [26]:
list(base_dir.glob("*.csv"))

[PosixPath('/content/drive/MyDrive/Research FakeNews/politifact_fake.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/churn_data.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/gossipcop_real.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/politifact_real.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/gossipcop_fake.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/politifact.csv'),
 PosixPath('/content/drive/MyDrive/Research FakeNews/politifact_agg.csv')]

In [27]:
df_fake = pd.read_csv(base_dir/'politifact_fake.csv')
df_real = pd.read_csv(base_dir/'politifact_real.csv')
df_fake['label']=0
df_real['label']=1
class_names = ["Fake", "Real"]

df_old = df_fake.append(df_real)
df_old.reset_index(drop=True, inplace=True)

In [28]:
len(df_old.id.unique()), len(df)

(1054, 1052)

Two ids appear in both fake and real. Exclude?

In [29]:
dup_ids, dup_groups = [], []
for n, g in df.groupby("id"):
    if len(g) > 1:
        dup_ids.append(n)
        dup_groups.append(g)
print(len(dup_ids))

0


In [30]:
dup_ids

[]

In [66]:
dup_groups

[]

In [33]:
dup_groups[1]

IndexError: ignored

In [34]:
df = df[df.id.map(lambda x: x not in dup_ids)]
df.reset_index(inplace=True)
df.shape

(1052, 17)

In [36]:
id2title = df_old[["id", "title"]].groupby("id").first().to_dict()["title"]

In [37]:
df.title.fillna("", inplace=True)
(df.title=="").sum()

0

In [38]:
for i, row in df.iterrows():
    if row.title == "":
        df.loc[i, "title"] = id2title.get(row.id, "")

In [39]:
(df.title=="").sum()

0

Is there a way to get content for those ids?

### Get texts

In [40]:
%%capture
!pip install -U selenium
!apt-get update && apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [41]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [42]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [43]:
df_old.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,0


In [44]:
id2url = df_old[["id", "news_url"]].groupby("id").first().to_dict()["news_url"]

In [45]:
for i, row in df[df.url.isna()].iterrows():
    df.loc[i, "url"] = id2url.get(row.id, "")

In [46]:
df_old.news_url.isna().sum()

61

In [47]:
(df.url.isna()).sum()

61

In [48]:
urls = df[df.text.isna()].url.tolist()

In [49]:
new_texts = dict()
for i, row in df[df.text.isna()].iterrows():
    url = row.url
    if (url is not None) and ("archive.org" in url) and (not url.endswith(".pdf")):
        print(url)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        _a = soup.find_all("div", {"id":"articleWrap"})
        text = None
        if len(_a):
            article = _a[0]
            paragraphs = article.find_all("p")
            text = "\n".join(p.text for p in paragraphs if len(p.text.strip()))
            text = " ".join(text.split())
        elif len(soup.find_all("meta", {"name":"DESCRIPTION"})):
            meta = soup.find_all("meta", {"name":"DESCRIPTION"})[0]
            text = meta.get("content", None)
            
        if text:
            new_texts[i] = text

https://web.archive.org/web/20070212105712/http://www.bea.gov:80/national/xls/gdpchg.xls
https://web.archive.org/web/20100202204116/http://alexander.senate.gov/public/index.cfm?p=NewsArticles


In [50]:
len(new_texts)

0

In [51]:
for t in new_texts.values():
    print(t[:100])

In [52]:
for i, t in new_texts.items():
    df.loc[i, "text"] = " ".join(text.split())

In [53]:
df.to_csv(base_dir/"politifact.csv")

## Data cleaning

In [54]:
df = df[df.title.str.len() > 0]

dups = []
for name, group in df.groupby("title"):
    if len(group) > 1:
        dups.append(group)

In [55]:
df = df.groupby("title", as_index=False).first()

texts = df.title + " " + df.text

In [56]:
len(texts)

894

In [57]:
texts

0                                                    NaN
1                                                    NaN
2                                                    NaN
3       If You Are Using This Toothpaste… Throw It Aw...
4      "Face the Nation" transcripts, August 26, 2012...
                             ...                        
889    ‘Smallville’ Star Confesses She Sold Children ...
890    ‘The Daily Show’s’ Michelle Wolf Lands Her Own...
891    “Dictionary” on President Obama’s Health Care ...
892    “Gay People Should Wear Specially-Colored Clot...
893                                                  NaN
Length: 894, dtype: object

## Source

In [60]:
source = df.url.str.replace("^(https://)?web.archive.org/web/\d+/", "", regex=True)

In [61]:
source = source.str.replace("^(https?://)?(www.)?", "", regex=True)

In [62]:
source = source.str.replace("\.(com|info|org|gov|tv|us|news|me|co.uk|net|club|co|live|edu|xyz|site|life|ru|online|tk|website|pw|one|world|mil).*$", "", regex=True)

In [63]:
len(source), len(source.unique())

(894, 519)

## Number of retweets

In [64]:
df["num_retweets"] = df.retweets.map(len)
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(len)
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

In [65]:
df.describe()

,level_0,index,summary,num_retweets,log_num_retweets,num_tweets,log_num_tweets
count,894.000000,894.000000,0.0,894.000000,894.000000,894.00000,894.000000
mean,542.328859,543.908277,NaN,557.787472,3.396452,66.61745,0.703731
std,311.964162,313.087083,NaN,2147.672525,2.716938,776.52660,1.763433
min,0.000000,0.000000,NaN,0.000000,0.000000,0.00000,0.000000
25%,262.500000,263.500000,NaN,1.000000,0.693147,0.00000,0.000000
50%,570.500000,571.500000,NaN,29.500000,3.417592,0.00000,0.000000
75%,814.750000,817.750000,NaN,264.500000,5.581610,0.00000,0.000000
max,1051.000000,1055.000000,NaN,29060.000000,10.277152,21984.00000,9.998116


## More to come